### Importation des librairies

In [2]:
from keras.applications import EfficientNetV2L
from keras.callbacks import EarlyStopping
from keras.layers import *
from keras.metrics import *

from tensorflow._api.v2.image import flip_left_right, adjust_brightness, adjust_contrast

import tensorflow as tf

import toolbox as tb

### Constantes

In [ ]:
EARLY_STOP = EarlyStopping(patience=3, monitor="val_loss")
INPUT_SHAPE = (224, 224, 3)
METRICS = ["accuracy"]
NEW_LAYERS = [
    GlobalAveragePooling2D(),
    # ------------------------- #
    Dense(512, activation="relu"),
    Dropout(0.2),
    # ------------------------- #
    Dense(512, activation="relu"),
    Dropout(0.2)
]

### Variables

In [ ]:
datasets_path = "data/images/"
nb_of_labels = len(tb.get_labels(datasets_path=datasets_path))
training_callbacks = [EARLY_STOP]

loss_function, model, train_dataset, training, validation_dataset = [None, None, None, None, None]

### Fonctions

#### Callback -- data augmentation

In [ ]:
def augment(image, label):
    image = flip_left_right(image) # rotation
    image = adjust_brightness(image, delta=0.1) # exposition
    image = adjust_contrast(image, contrast_factor=1.75) # contraste

    return (image, label)

### Création du modèle

#### Création des datasets

Dataset -- entrainement : **80%** de l'intégralité des images

Dataset -- validation : **20%** de l'intégralité des images

In [ ]:
train_dataset, validation_dataset = tb.get_datasets(folders_path=datasets_path)

#### Instanciation d'un nouveau modèle

In [ ]:
model = tb.new_model(EfficientNetV2L, include_top=False, input_shape=INPUT_SHAPE)

#### Gel des couches du modèle instancié

In [ ]:
tb.freeze(model)

#### Ajout de nouvelles couches convolutives

> Les nouvelles couches ajoutées seront les seules couches à être entrainées. Le reste des couches ont été gelées précédemment.

In [ ]:
model = tb.new_cnn(model, nb_of_labels=nb_of_labels, new_layers=NEW_LAYERS)

#### Mise en place de la data augmentation

La `data augmentation` va permettre d'augmenter le dataset d'entrainement avec des images qui vont être :

- orientées de manières diverses
- élargies ou reduites au travers de zooms
- ...

In [ ]:
train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)

#### Prefetch

Mise en place de prefetch sur le dataset d'entrainement et le dataset de validation.

Ces prefetch vont permettre d'optimiser l'entrainement du modèle.

In [ ]:
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)

#### Compilation du modèle

In [ ]:
loss_function = tb.get_loss_function(nb_of_labels=nb_of_labels)

tb.compile_model(model, loss=loss_function, metrics=METRICS, learning_rate=0.001)

#### Entrainement du modèle

In [ ]:
training = tb.train_model(
    model,
    train_data = train_dataset,
    validation_data = validation_dataset,
    nb_of_epochs = 20,
    callbacks = training_callbacks
)

#### Graphiques récapitulatifs de l'entrainement

##### Accuracy

In [ ]:
tb.graph("accuracy", training).show()

##### Loss

In [ ]:
tb.graph("loss", training).show()

### Sauvegarde du modèle

In [ ]:
model.save(f"mes_modeles/mon_modele__{NB_OF_LABELS}_labels")